<a href="https://colab.research.google.com/github/Uthmanic/DSN-AI-Bootcamp-2020/blob/master/Copy_of_DSN_Zindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
%matplotlib inline

In [3]:
train = pd.read_csv('https://raw.githubusercontent.com/Uthmanic/DSN-AI-Bootcamp-2020/master/Train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Uthmanic/DSN-AI-Bootcamp-2020/master/Test.csv')
train_copy = train.copy()
test_copy = test.copy()

In [4]:
#train.info()

In [5]:
#train_copy.isnull().sum()

In [6]:
#test_copy.isnull().sum()

In [7]:
test_copy.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,0.0,1649749.0,5446.0,0.0,48187375,114686147.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1086.0,6204.0,2676.0,60.0,14691.0,9794.0,NaN,238.0,1.8334,0.0,NaN,22.0,2.0,14.0,0.0,NaN,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,18308285.0,622789.0,1025793.0,1396510.0,34694910,87479487.0,1.0,1.0,1.0,2.0,2.0,142.560,NaN,NaN,570.585,3082.5,10221.0,6814.0,60.0,11955.0,3528.0,NaN,1216.0,19.5000,3.0,NaN,30.0,18.0,14.0,2.0,NaN,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,0.0,118256.0,NaN,NaN,21973443,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852.0,4836.0,NaN,NaN,NaN,7.8334,0.0,NaN,2.0,NaN,0.0,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,493641.0,1391064.0,1176725.0,1220293.0,16576457,NaN,2.0,2.0,2.0,3.0,3.0,109.005,13050.00,NaN,109.005,1414.5,4197.0,2798.0,60.0,14964.0,NaN,NaN,248.0,9.0000,1.0,1.0,36.0,0.0,26.0,0.0,NaN,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,5752921.0,271133.0,613064.0,4779643.0,48187375,72049802.0,0.0,0.0,0.0,3.0,3.0,139.065,469.29,469.29,15.180,2724.0,9855.0,6570.0,182.0,11679.0,5900.0,NaN,2388.0,16.6666,1.0,0.0,24.0,28.0,20.0,0.0,NaN,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [8]:
train_copy.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [9]:
train_copy = train_copy.fillna(-1)
test_copy = test_copy.fillna(-1)

In [10]:
#train_copy.isnull().sum()

In [11]:
#test_copy.isnull().sum()

In [12]:
from sklearn.preprocessing import LabelEncoder
 
train_copy["form_field47"] = LabelEncoder().fit_transform(train_copy["form_field47"])
train_copy["default_status"] = LabelEncoder().fit_transform(train_copy["default_status"])
test_copy["form_field47"] = LabelEncoder().fit_transform(test_copy["form_field47"])

In [13]:
train_copy.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,-1.00,-1.00,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,-1.0,1052.0,8.0000,2.0,-1.0,12.0,12.0,4.0,0.0,-1.0000,-1.0,0.392854,2.02,0.711632,0.0,0.0,0,-1.000000,1.129518,0.044335,0
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,-1.0,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,-1.0000,-1.0,0.314281,8.08,0.183584,-1.0,0.0,0,349.805730,1.620483,0.322436,0
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,-1.0,956940.0,-1.0,192944.0,1079864.0,-1.0,-1.0,0.0,16576457,22452691.0,-1.0,-1.0,-1.0,0.0,0.0,-1.000,-1.00,-1.00,-1.000,1597.5,4746.0,3164.0,60.0,15696.0,-1.0,14508.0,70.0,0.6666,0.0,-1.0,12.0,0.0,6.0,0.0,149.6805,-1.0,0.162965,18.18,0.791136,0.0,0.0,0,-1.000000,1.513370,0.011640,1
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,-1.0,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,-1.0,-1.0,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,-1.0000,-1.0,0.488884,2.02,0.685168,-1.0,0.0,0,89.940100,0.664452,0.082729,0
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,-1.0,0.275000,12.12,0.438168,0.0,0.0,0,97.887502,1.427891,0.045630,0


In [14]:
#model feature and target
X = train_copy.drop(["Applicant_ID", "default_status","form_field17"], axis =1)
y = train_copy["default_status"]
 
test_data = test_copy.drop(["Applicant_ID","form_field17"], axis = 1)

In [15]:
!pip install catboost

In [16]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
 
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=1000,eval_metric='AUC',max_depth=4,learning_rate=0.1,od_wait=50, reg_lambda=3,
                          bootstrap_type='Bayesian')
gbc = GradientBoostingClassifier(verbose=100, random_state=101)
etc = ExtraTreesClassifier(verbose=100, random_state=101)
lgbm = LGBMClassifier(random_state=101,verbose=100)
catb = CatBoostClassifier(n_estimators=1000,eval_metric='AUC',max_depth=4,learning_rate=0.1,od_wait=50, reg_lambda=3,
                          bootstrap_type='Bayesian')

In [18]:
new = []
predi = []
 
fold = StratifiedKFold(n_splits=55, shuffle=True, random_state=101)
 
for train_index, test_index in fold.split(X,y):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y[train_index],y[test_index]
 
  catb = CatBoostClassifier(n_estimators=1000,eval_metric='AUC',max_depth=4,learning_rate=0.1,od_wait=50, reg_lambda=3,
                          bootstrap_type='Bayesian',use_best_model=True)
 
  catb.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_test,y_test)],
           early_stopping_rounds=100,verbose=100)
  
  catb_pred = catb.predict_proba(X_test)[:,1]
 
  new.append(roc_auc_score(y_test,catb_pred))
 
  catpred = catb.predict_proba(test_data)[:,1]
  predi.append(catpred)

0:	test: 0.7669426	test1: 0.7463693	best: 0.7463693 (0)	total: 33.4ms	remaining: 33.4s
100:	test: 0.8405282	test1: 0.8266892	best: 0.8266892 (100)	total: 3.8s	remaining: 33.9s
200:	test: 0.8472583	test1: 0.8297633	best: 0.8299506 (190)	total: 7.26s	remaining: 28.8s
300:	test: 0.8520874	test1: 0.8294148	best: 0.8302159 (206)	total: 10.7s	remaining: 24.9s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8302158648
bestIteration = 206

Shrink model to first 207 iterations.
0:	test: 0.7803983	test1: 0.7806190	best: 0.7806190 (0)	total: 34.8ms	remaining: 34.7s
100:	test: 0.8400639	test1: 0.8391261	best: 0.8391261 (100)	total: 3.56s	remaining: 31.7s
200:	test: 0.8470156	test1: 0.8419142	best: 0.8422835 (181)	total: 7.01s	remaining: 27.9s
300:	test: 0.8517471	test1: 0.8419350	best: 0.8426372 (286)	total: 10.5s	remaining: 24.3s
400:	test: 0.8556754	test1: 0.8421014	best: 0.8428088 (319)	total: 13.9s	remaining: 20.8s
Stopped by overfitting detector  (100 iterations wait)

be

In [19]:
lgbm_new = []
lgbm_predi = []
 
fold = StratifiedKFold(n_splits=55, shuffle=True, random_state=101)
 
for train_index, test_index in fold.split(X,y):
  x_train, x_test = X.iloc[train_index], X.iloc[test_index]
  Y_train, Y_test = y[train_index],y[test_index]
 
  lgbm = LGBMClassifier(n_estimators=1000,eval_metric='AUC',max_depth=4,learning_rate=0.1,od_wait=50, reg_lambda=3,
                          bootstrap_type='Bayesian',use_best_model=True)
 
  lgbm.fit(x_train, Y_train, eval_set=[(x_train,Y_train),(x_test,Y_test)],
           early_stopping_rounds=100,verbose=100)
  
  lgbm_pred = lgbm.predict_proba(x_test)[:,1]
 
  lgbm_new.append(roc_auc_score(Y_test,lgbm_pred))
 
  lgbm_pred = lgbm.predict_proba(test_data)[:,1]
  lgbm_predi.append(lgbm_pred)

Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.391484	valid_1's binary_logloss: 0.418174
[200]	training's binary_logloss: 0.376886	valid_1's binary_logloss: 0.41716
[300]	training's binary_logloss: 0.365279	valid_1's binary_logloss: 0.418384
Early stopping, best iteration is:
[224]	training's binary_logloss: 0.373941	valid_1's binary_logloss: 0.416113
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.391638	valid_1's binary_logloss: 0.402562
[200]	training's binary_logloss: 0.376815	valid_1's binary_logloss: 0.400112
[300]	training's binary_logloss: 0.365117	valid_1's binary_logloss: 0.401441
Early stopping, best iteration is:
[238]	training's binary_logloss: 0.371866	valid_1's binary_logloss: 0.39956
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.390927	valid_1's binary_logloss: 0.407442
Early stopping, best iteration is:
[86]	training's

In [20]:

xgb_new = []
xgb_predi = []
 
fold = StratifiedKFold(n_splits=55, shuffle=True, random_state=101)
 
for train_index, test_index in fold.split(X,y):
  xtrain, xtest = X.iloc[train_index], X.iloc[test_index]
  Ytrain, Ytest = y[train_index],y[test_index]
 
  xgb = XGBClassifier(n_estimators=1000,max_depth=4,learning_rate=0.1,od_wait=50, reg_lambda=3,
                          bootstrap_type='Bayesian')
 
  xgb.fit(xtrain, Ytrain, eval_set=[(xtrain,Ytrain),(xtest,Ytest)],
           early_stopping_rounds=100,verbose=100)
  
  xgb_pred = xgb.predict_proba(xtest)[:,1]
 
  xgb_new.append(roc_auc_score(Ytest,xgb_pred))
 
  xgb_pred = xgb.predict_proba(test_data)[:,1]
  xgb_predi.append(xgb_pred)

[0]	validation_0-error:0.211364	validation_1-error:0.238469
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 100 rounds.
[100]	validation_0-error:0.181736	validation_1-error:0.202159
[200]	validation_0-error:0.172132	validation_1-error:0.208047
Stopping. Best iteration:
[101]	validation_0-error:0.181354	validation_1-error:0.200196

[0]	validation_0-error:0.211127	validation_1-error:0.201178
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 100 rounds.
[100]	validation_0-error:0.181572	validation_1-error:0.178606
[200]	validation_0-error:0.17266	validation_1-error:0.175662
[300]	validation_0-error:0.165857	validation_1-error:0.177625
Stopping. Best iteration:
[218]	validation_0-error:0.171441	validation_1-error:0.174681

[0]	validation_0-error:0.211946	validation_1-error:0.207066
Multiple 

In [21]:
# accuracy of test set
print('lgbm accuracy: {}'.format(np.mean(lgbm_new)))
print('catb accuracy: {}'.format(np.mean(new)))
print('xgb accuracy: {}'.format(np.mean(xgb_new)))


lgbm accuracy: 0.8404975754344365
catb accuracy: 0.8417343891466088
xgb accuracy: 0.836088251648573


%%capture
# predict test set
lgbm_pred = lgbm.predict_proba(test_data)[:,1]
etc_pred = etc.predict_proba(test_data)[:,1]
catb_pred = catb.predict_proba(test_data)[:,1]
gbc_pred = gbc.predict_proba(test_data)[:,1]

In [22]:
from google.colab import files
uploaded = files.upload()

Saving 64_submission.csv to 64_submission (1).csv


In [23]:
import io
sub64 = pd.read_csv(io.BytesIO(uploaded['64_submission.csv']))

In [24]:
weighted_average = (((catpred * 0.4 + sub64.default_status * 0.6) * 0.6) + (lgbm_pred * 0.3) + (xgb_pred * 0.1))

In [ ]:
weighted_average.

In [ ]:
pwd

In [27]:
pd.DataFrame({"Applicant_ID": test_copy["Applicant_ID"], "default_status": weighted_average}).to_csv("80_submission.csv", index = False)

In [28]:
from google.colab import files
files.download('80_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>